In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import librosa as lb
import soundfile as sf
import pandas as pd
import cv2
from pathlib import Path
import re,sys

import torch
from torch import nn
from  torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

import time

sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

# Configs

In [ ]:
NUM_CLASSES = 397
SR = 32_000
DURATION = 5
THRESH = 0.6


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

TEST_AUDIO_ROOT = Path("../input/birdclef-2021/test_soundscapes")
SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
TARGET_PATH = None
    
if not len(list(TEST_AUDIO_ROOT.glob("*.ogg"))):
    TEST_AUDIO_ROOT = Path("../input/birdclef-2021/train_soundscapes")
    SAMPLE_SUB_PATH = None
    # SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
    TARGET_PATH = Path("../input/birdclef-2021/train_soundscape_labels.csv")

# Data

In [ ]:
class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin, fmax, **kwargs):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs["n_fft"] = kwargs.get("n_fft", self.sr//10)
        kwargs["hop_length"] = kwargs.get("hop_length", self.sr//(10*4))
        self.kwargs = kwargs

    def __call__(self, y):

        melspec = lb.feature.melspectrogram(
            y, sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax, **self.kwargs,
        )

        melspec = lb.power_to_db(melspec).astype(np.float32)
        return melspec

In [ ]:
def mono_to_color(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V

def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, length - np.zeros(len(y))])
    elif len(y) > length:
        y = y[:length]
    return y

class BirdCLEFDataset(Dataset):
    def __init__(self, data, sr=SR, n_mels=128, fmin=0, fmax=None, duration=DURATION, step=None, res_type="kaiser_fast", resample=True):
        
        self.data = data
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2

        self.duration = duration
        self.audio_length = self.duration*self.sr
        self.step = step or self.audio_length
        
        self.res_type = res_type
        self.resample = resample

        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin,
                                                 fmax=self.fmax)
    def __len__(self):
        return len(self.data)
    
    @staticmethod
    def normalize(image):
        image = image.astype("float32", copy=False) / 255.0
        image = np.stack([image, image, image])
        return image
    
    def audio_to_image(self, audio):
        melspec = self.mel_spec_computer(audio) 
        image = mono_to_color(melspec)
        image = self.normalize(image)
        return image

    def read_file(self, filepath):
        audio, orig_sr = sf.read(filepath, dtype="float32")

        if self.resample and orig_sr != self.sr:
            audio = lb.resample(audio, orig_sr, self.sr, res_type=self.res_type)
          
        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])
            
        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]
            
        images = [self.audio_to_image(audio) for audio in audios]
        images = np.stack(images)
        
        return images
    
        
    def __getitem__(self, idx):
        return self.read_file(self.data.loc[idx, "filepath"])

In [ ]:
data = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(TEST_AUDIO_ROOT).glob("*.ogg")],
    columns = ["filename", "id", "site", "date", "filepath"]
)
print(data.shape)
data.head()

In [ ]:
df_train = pd.read_csv("../input/birdclef-2021/train_metadata.csv")

LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df_train["primary_label"].unique()))}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}

# Inference

In [ ]:
test_data = BirdCLEFDataset(data=data)
len(test_data), test_data[0].shape

In [ ]:
class Model_eff2(nn.Module):
    def __init__(self,name,pretrained=True):
        super(Model_eff2, self).__init__()
        self.eff = timm.create_model(name,pretrained=pretrained)
        #nb_ft = self.eff.fc.in_features
        #nb_ft = self.eff.classifier.in_features
        #self.eff.fc = nn.Linear(nb_ft, CFG.CLASS_NUM)
        self.fc = nn.Linear(1000, NUM_CLASSES)
        #self.eff.classifier = nn.Linear(nb_ft, CFG.CLASS_NUM)
        #self._init_params()    
    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)

    def forward(self, x):
        x = self.eff(x)
        x = self.fc(x)
        return x

def load_net(name,checkpoint_path, num_classes=NUM_CLASSES):
    #net = Model_eff('efficientnet_b0',pretrained=False)
    net = Model_eff2(name,pretrained=False)
    #net.fc = nn.Linear(net.fc.in_features, num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    for key in list(d.keys()):
        d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net

In [ ]:
@torch.no_grad()
def get_thresh_preds(out, thresh=None):
    thresh = thresh or THRESH
    o = (-out).argsort(1)
    npreds = (out > thresh).sum(1)
    preds = []
    for oo, npred in zip(o, npreds):
        preds.append(oo[:npred].cpu().numpy().tolist())
    return preds

def get_bird_names(preds):
    bird_names = []
    for pred in preds:
        if not pred:
            bird_names.append("nocall")
        else:
            bird_names.append(" ".join([INV_LABEL_IDS[bird_id] for bird_id in pred]))
    return bird_names

def predict(nets, test_data, names=True):
    preds = []
    with torch.no_grad():
        for idx in  tqdm(list(range(len(test_data)))):
            xb = torch.from_numpy(test_data[idx]).to(DEVICE)
            pred = 0.
            for i,net in enumerate(nets):
                o = net(xb)
                o = torch.sigmoid(o)

                pred += o

            pred /= len(nets)
            
            if names:
                pred = get_bird_names(get_thresh_preds(pred))

            preds.append(pred)
    return preds

def preds_as_df(data, preds):
    sub = {
        "row_id": [],
        "birds": [],
    }
    
    for row, pred in zip(data.itertuples(False), preds):
        row_id = [f"{row.id}_{row.site}_{5*i}" for i in range(1, len(pred)+1)]
        sub["birds"] += pred
        sub["row_id"] += row_id
        
    sub = pd.DataFrame(sub)
    
    if SAMPLE_SUB_PATH:
        sample_sub = pd.read_csv(SAMPLE_SUB_PATH, usecols=["row_id"])
        sub = sample_sub.merge(sub, on="row_id", how="left")
        sub["birds"] = sub["birds"].fillna("nocall")
    return sub

def get_metrics(s_true, s_pred):
    s_true = set(s_true.split())
    s_pred = set(s_pred.split())
    n, n_true, n_pred = len(s_true.intersection(s_pred)), len(s_true), len(s_pred)
    
    prec = n/n_pred
    rec = n/n_true
    f1 = 2*prec*rec/(prec + rec) if prec + rec else 0
    
    return {"f1": f1, "prec": prec, "rec": rec, "n_true": n_true, "n_pred": n_pred, "n": n}

In [ ]:
checkpoint_paths0 = [Path("../input/birdclefweights/resnest50d_f1.0741.pt"),
                     Path("../input/birdclefweights/ecaresnet101d_acc0.731.pt"),
                     Path("../input/birdclefweights/eca_nfnet_l1_acc0.720.pt")
                    ]

names0 = ["resnest50d",
          "ecaresnet101d",
          "eca_nfnet_l1",
         ]

nets0 = [
        load_net(name,checkpoint_path.as_posix()) for name,checkpoint_path in zip(names0,checkpoint_paths0) 
]
                     

model =torch.load("../input/birdclefweights/efficientnetv2_rw_m_0.716.pth")
model.load_state_dict(torch.load("../input/birdclefweights/efficientnetv2_rw_m_0.721.pt"))
nets0.append(model)

checkpoint_paths1 = [
    
    Path("../input/birdclefweights/ecaresnext50t_32x4d_acc0.719.pt"),
    Path("../input/birdclefweights/ecaresnet50d_f10.719_focal.pt"),
    Path("../input/birdclefweights/efficientnetv2_rw_s_acc0.713.pt"),
    Path("../input/birdclefweights/ecaresnet200d_acc0.714.pt"),
    
]

names1 = [
          "ecaresnext50t_32x4d",
          "ecaresnet50d",
          "efficientnetv2_rw_s",
          "ecaresnet200d",
]

nets1 = [
        load_net(name,checkpoint_path.as_posix()) for name,checkpoint_path in zip(names1,checkpoint_paths1) 
]

checkpoint_paths2 = [
    #Path("../input/birdclefweights/eca_nfnet_l0_acc0.705.pt"),
    Path("../input/birdclefweights/resnest50d_acc0.712.pt"),
    Path("../input/birdclefweights/ecaresnet101d_f10.696.pt"),
    Path("../input/birdclefweights/ecaresnet50d_acc0.711.pt"),
    Path("../input/birdclefweights/ecaresnet50d_acc0.706.pt"),
    Path("../input/birdclefweights/eca_nfnet_l0_acc0.705.pt"),
    Path("../input/birdclefweights/resnest50d_4s2x40d_acc0.710.pt"),
]

names2 = ["resnest50d",
          "ecaresnet101d",
          "ecaresnet50d",
          "ecaresnet50d",
          "eca_nfnet_l0",
          "resnest50d_4s2x40d",          
]


nets2 = [
        load_net(name,checkpoint_path.as_posix()) for name,checkpoint_path in zip(names2,checkpoint_paths2) 
]



In [ ]:
weight = torch.tensor([1,1,1,1,1,1])
weight = weight*(len(weight)/weight.sum())
pred_probas0 = predict(nets0, test_data, names=False)
pred_probas1 = predict(nets1, test_data, names=False)
pred_probas2 = predict(nets2, test_data, names=False)
pred_pro = [(pred0 + pred1+0.4*pred2)/2.4 for pred0, pred1,pred2 in zip(pred_probas0, pred_probas1,pred_probas2)]

In [ ]:
for THRESH in [0.205]:
    preds = [get_bird_names(get_thresh_preds(pred, thresh=THRESH)) for pred in pred_pro]
    # preds[:2]
    sub = preds_as_df(data, preds)
    print(sub.shape)
    sub.to_csv("submission.csv", index=False)
    if TARGET_PATH:
        sub_target = pd.read_csv(TARGET_PATH)
        sub_target = sub_target.merge(sub, how="left", on="row_id")

        print(sub_target["birds_x"].notnull().sum(), sub_target["birds_x"].notnull().sum())
        assert sub_target["birds_x"].notnull().all()
        assert sub_target["birds_y"].notnull().all()

        df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])
        print(f"thre:{THRESH}")
        print(df_metrics.mean())

In [ ]:
sub